In [10]:
import bokeh
from bokeh.io import output_file, show
from bokeh.layouts import row,column
from bokeh.plotting import figure,output_file, save, show, ColumnDataSource
from bokeh.models.tools import HoverTool
from collections import OrderedDict
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline

In [11]:
#El formato del archivo de entrada hay que adaptarlo a como quede el archivo descargado de Kbotoolbox.
# De todas maneras el nombre de las columnas seria el mismo
data= pd.read_csv(r".\Grafico_baliza.csv",sep=";",
                  encoding = "latin")
data

,Fecha,Hora,Pais,Grupo,cm,Seccion,Nombre
0,16-01-2021,16:00,Chile,Guía,20,A,Maqui
1,17-01-2021,17:00,Chile,Turista,50,A,Raulí Bellota
2,18-01-2021,18:00,Alemania,Turista,120,A,Maria Jimena
3,19-01-2021,13:00,Chile,Guía,180,A,Erick Calvo
4,20-01-2021,12:00,Israel,Turista,10,B,Maria verdana
5,21-01-2021,18:00,Canadá,Turista,40,B,Rora Cursiva
6,22-01-2021,13:00,Brasil,Guía,100,B,Rata blanca
7,23-01-2021,12:00,Chile,Guía,150,B,Eureka Vin
8,24-01-2021,18:00,Chile,Turista,190,B,Chancado de roca
9,25-01-2021,13:00,Chile,Turista,5,C,Atención poblador


In [12]:
#Transformando fecha y hora a datetime y eliminando nans
data['date']=pd.to_datetime(data['Fecha'], format="%d-%m-%Y").dt.date.astype(str)
data['time']=pd.to_datetime(data['Hora'], format="%H:%M").dt.time.astype(str)
data["date_time"]=pd.to_datetime(data['Fecha']+" "+data['Hora'])
data.set_index('date_time', inplace=True)
data.drop(["Fecha","Hora"],axis=1,inplace=True)
data.dropna(inplace=True);
data

,Pais,Grupo,cm,Seccion,Nombre,date,time
date_time,,,,,,,
2021-01-16 16:00:00,Chile,Guía,20,A,Maqui,2021-01-16,16:00:00
2021-01-17 17:00:00,Chile,Turista,50,A,Raulí Bellota,2021-01-17,17:00:00
2021-01-18 18:00:00,Alemania,Turista,120,A,Maria Jimena,2021-01-18,18:00:00
2021-01-19 13:00:00,Chile,Guía,180,A,Erick Calvo,2021-01-19,13:00:00
2021-01-20 12:00:00,Israel,Turista,10,B,Maria verdana,2021-01-20,12:00:00
2021-01-21 18:00:00,Canadá,Turista,40,B,Rora Cursiva,2021-01-21,18:00:00
2021-01-22 13:00:00,Brasil,Guía,100,B,Rata blanca,2021-01-22,13:00:00
2021-01-23 12:00:00,Chile,Guía,150,B,Eureka Vin,2021-01-23,12:00:00
2021-01-24 18:00:00,Chile,Turista,190,B,Chancado de roca,2021-01-24,18:00:00


In [13]:
#Sumando largo de balizas anteriores para ver pérdida total de hielo en el periodo
baliza= []
for i in range(data.shape[0]):
    if data["Seccion"].iloc[i] == "A":
        baliza.append(data["cm"].iloc[i])
    elif data["Seccion"].iloc[i] == "B":
        baliza.append(data["cm"].iloc[i]+200)
    elif data["Seccion"].iloc[i] == "C":
        baliza.append(data["cm"].iloc[i]+400)
    elif data["Seccion"].iloc[i] == "D":
        baliza.append(data["cm"].iloc[i]+600)
        
data["baliza"]= baliza
    

In [26]:
#Graficando con Bokeh

source=  ColumnDataSource(ColumnDataSource.from_df(data))
#Figura width=750, plot_height=250,

fig = figure(width=750, plot_height=250, title=None,
             x_axis_type = "datetime",
             tools= 'pan,box_zoom,save,reset,hover' , sizing_mode='scale_width')

#Gráfico línea
fig.line(x="date_time",y="baliza",color='dodgerblue',line_width = 2,
         legend_label= "Emergencia de baliza(cm)",source=source)

#Puntito sobre cada dato
fig.circle(x="date_time",
           y="baliza",
           source=source,
           color='orangered',
           selection_color='deepskyblue',
           nonselection_color='lightgray',
           nonselection_alpha=0.3,
          size=8)

#Esto no está funcionando la idea es que se desplieguen estos datos cuando pinchas uno de los puntos
# Format tooltip
tooltips = [('Seccion', '@Seccion'),
            ('Pais', '@Pais'),
            ('Nombre', '@Nombre'),
            ('Grupo','@Grupo')
]
#tooltips = """

#    <div>
#        <h3> Baliza: @Seccion </h3>
#        <div> <strong>Pais:</strong> @Pais</div>
#       <div> <strong>Nombre </strong>@Nombre</div>
#       <div> <strong>Grupo</strong>@Grupo </div>
#   </div>
#
#""

#Labels
fig.xaxis.axis_label = "Fecha"
fig.yaxis.axis_label = "Pérdida de hielo (cm)"
#Leyenda
fig.legend.location = "top_left"
fig.legend.click_policy="hide"
fig.legend.padding=5
fig.legend.margin=0

#Circulo que aparece al acercarse al dato
hover_glyph = fig.circle(x='date_time', y='baliza', source=source,
                        size=15, alpha=0,
                       hover_fill_color='orangered', hover_alpha=0.5)

#Tools
#fig.add_tools(HoverTool(tooltips=tooltips, renderers=[hover_glyph]))
#fig.hover = HoverTool(tooltips=tooltips, renderers=[hover_glyph])

fig.hover.tooltips = tooltips


output_file(r".\Grafico_baliza.html")
show(fig)

In [20]:
with open('.\Grafico_baliza.html') as links_file:
    content = links_file.read()
    uno = content[180:500]
    dos = content[600:len(content)-20]
    
    #linkparser = LinkParser()
    #a = linkparser.feed(content) 

In [23]:
bloque_uno = """
    {% extends "Frontal/layout.html" %}
    {% block head %}
    """

bloque_dos = """
    {% endblock %}

    {% block content %}
    
    
    <div class="container-fluid pb-4 pt-4 paddding">
      <div class="container paddding">
          <div class="row mx-0">
              <div class="col-md-8 animate-box" data-animate-effect="fadeInLeft">
                  <div>
                      <h2 class="fh5co_heading fh5co_heading_border_bottom py-2 mb-4">Prueba</h2>
                  </div>
              </div>
          </div>
      </div>
    </div>
    
              <div class="row mx-0 ">
            <div class="col-md-6 animate-box" data-animate-effect="fadeInLeft">
                <h5>En los últimos 7 días, el glaciar Exploradores ha perdido en total "Numero" cm de espesor.</h5>
            </div>
            <div class="col-md-6 animate-box" style="background-color: aliceblue" data-animate-effect="fadeInLeft">
                <h5 style="transform:translateY(50%)">Cantidad de mediciones "+numero+"</h5>
            </div>
          </div>
          <div class="row  mx-0 mt-3 mb-3">
            <div class=" col-md-6 animate-box" data-animate-effect="fadeInLeft">
              <div>
                <h5>Gráficos de nacionalidades"</h5>
                <div class="bk-root escritorio" 
    """

bloque_movil = """
<div class="bk-root movil" 
"""

bloque_tablet = """
<div class="bk-root tablet" 
"""

bloque_escritorio = """
<div class="bk-root escritorio" 
"""


## Deben ir las fotos que se recolectan.
bloque_tres = """
    {% endblock %} 
    """

In [24]:
print(bloque_uno + uno + bloque_dos + dos + bloque_tres)


    {% extends "Frontal/layout.html" %}
    {% block head %}
          <script type="text/javascript" src="https://cdn.bokeh.org/bokeh/release/bokeh-2.1.1.min.js" integrity="sha384-kLr4fYcqcSpbuI95brIH3vnnYCquzzSxHPU6XGQCIkQRGJwhg0StNbj1eegrHs12" crossorigin="anonymous"></script>
        <script type="text/javascript">
            Bokeh.set_log_level("info");
        </script>
     
    {% endblock %}

    {% block content %}
    
    
    <div class="container-fluid pb-4 pt-4 paddding">
      <div class="container paddding">
          <div class="row mx-0">
              <div class="col-md-8 animate-box" data-animate-effect="fadeInLeft">
                  <div>
                      <h2 class="fh5co_heading fh5co_heading_border_bottom py-2 mb-4">Prueba</h2>
                  </div>
              </div>
          </div>
      </div>
    </div>
    
              <div class="row mx-0 ">
            <div class="col-md-6 animate-box" data-animate-effect="fadeInLeft">
                <h5>

In [25]:
doc = bloque_uno + uno + bloque_dos + dos + bloque_tres
f = open('prueba.html','w')

f.write(doc)
f.close()